In [ ]:
%cd '/content/drive/MyDrive/UNI/AN2DL/Homework2/data'

## Import libaries

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import random
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import math

## Split train test

In [ ]:
x = np.load('x_train.npy')
y = np.load('y_train.npy')

x, y = shuffle(x, y, random_state=0)

split = math.floor(len(x)*0.75)

x_train, x_val = x[:split], x[split:]
y_train, y_val = y[:split], y[split:]

## Visualize data

### Histogram of the classes

In [ ]:
plt.figure(figsize=(15,10))
plt.hist(y,bins=12), plt.hist(y_train,bins=12), plt.hist(y_val,bins=12)
plt.legend(['Full Dataset', 'Train', 'Validation'], prop={'size': 20})

### Random Samples

In [ ]:
sample = random.randint(0,split)
pd.DataFrame(x[sample]).plot()
print(sample)
print(y_train[sample])

In [ ]:
fig, axes = plt.subplots(20,figsize=(10,20))
a = np.where(y==11)[0]
for i, ax in enumerate(axes):
    ax.plot((x[a[i]]))

## Build the Model

In [ ]:
x_mean = x_train.mean()
x_var = x_train.var()

In [ ]:
input_shape = x_train.shape[1:]

In [ ]:
model = keras.Sequential([
    layers.Input(input_shape),

    layers.Normalization(mean=x_mean, variance=x_var),

    layers.Conv1D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.3),
    
    layers.Conv1D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.3),

    layers.Conv1D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling1D(),
    layers.Dropout(0.3),

    layers.Bidirectional(layers.LSTM(100, recurrent_regularizer=keras.regularizers.L1L2(1e-2, 1e-2))),
    layers.Dropout(0.5),

    layers.Dense(12, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model)

## Training

In [ ]:
batch_size = 64
epochs = 100
patience = 10

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(), optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True),
                 keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)]
).history

In [ ]:
model.save('SubmissionModel')

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history['loss'], label='Train', color='red')
plt.plot(history['val_loss'], label='Validation', color='blue')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], label='Train', color='red')
plt.plot(history['val_accuracy'], label='Validation', color='blue')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()